# PROJECT GEGEVENSBANKEN - SQL MODELOPLOSSINGEN

# Imports

In [1]:
# Setup
import getpass  # Package om een paswoordveldje te genereren.
import json  # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import mysql.connector  # MySQL package
import numpy as np
import os
import pandas as pd  # Populaire package voor data-verwerking
import re
import sys

from inspect import getsource
from IPython.display import display, Markdown
from os.path import dirname

# Imports van binnen onze eigen repository
this_dir = os.getcwd()
root_dir = dirname(this_dir)
docs_dir = os.path.join(root_dir, "docs")
source_dir = os.path.join(root_dir, "src")
script_dir = os.path.join(root_dir, "scripts")
solution_dir = os.path.join(root_dir, "solution")

sys.path.append(script_dir)
sys.path.append(source_dir)

import db_project  # onze eigen codebase in deze repository
from db_project import (
    verbind_met_GB,
    run_query,
    res_to_df,
)  # 3 basic functionalities, explained in introduction.

# Text files from which some descriptions are rendered (this helps us to keep everything perfectly consistent across different notebooks etc)
introduction_file = os.path.join(docs_dir, "01-introduction.md")
instructions_file = os.path.join(docs_dir, "02-instructions.md")
qry_description_file = os.path.join(docs_dir, "03-qry-descriptions.md")
submissions_file = os.path.join(docs_dir, "04-submissions.md")

from db_project import parse_markdown

In [2]:
# Imports specific to the solution
model_solution_dir = os.path.join(root_dir, "do-not-release-until-ok")
sys.path.append(model_solution_dir)
import model_solution as ms # Importeer de file `solution/model_solution.py` als module 'ms'
from db_project import collect_relevant_solutions

In [3]:
display(Markdown(parse_markdown(introduction_file, section_number=[1,2, 3]))) # Tekst en uitleg worden ingelezen uit bestanden uit de `docs` folder.

# 1. Inleiding

Cf. het document `prerequisites.pdf` in de `docs` folder. 

## 1.1 Python packages

De import statements (cf. boven) vormen de standaardconfiguratie van python-packages die we gebruiken. _Van deze configuratie afwijken is dan ook op eigen risico, gezien dat betekent dat je ook afwijkt van de configuratie van de computer waarop wij jullie oplossingen runnen._

## 1.2 Interageren met een gegevensbank

Naast de standaard packages, importeren we ook drie zelfgemaakte functies. Deze implementeren functionaliteiten die je vaak zal nodig hebben als je vanuit python queries naar je gegevensbank wil sturen: `verbind_met_GB`, `run_query` en `res_to_df`. Hun source code (en documentatie) bekijken geeft de nodige informatie over hoe ze te gebruiken:
    
    


In [4]:
print(getsource(verbind_met_GB))
print('- --- - --- - --- - --- - --- - --- - - --- - --- - --- - --- - --- - --- -\n')
print(getsource(run_query))
print('- --- - --- - --- - --- - --- - --- - - --- - --- - --- - --- - --- - --- -\n')
print(getsource(res_to_df))

def verbind_met_GB(username, hostname, gegevensbanknaam, password=None):
    """
    Maak verbinding met een externe gegevensbank

    Parameters
    ----------

    username:           str
                        Username van de gebruiker
    hostname            str
                        Naam van de host. In het geval van lokale server 'localhost'
    gegevensbanknaam    str
                        Naam van de gegevensbank
    password            str, None
                        Wachtwoord kan al meegegeven worden. Indien niet, wordt
                        een wachtwoordveldje gegenereerd waar de gebruiker het
                        kan ingeven.
    Returns
    -------
    connection          connection object
                        Dit is het soort object dat wordt teruggeven door
                        connect() methods van packages die voldoen aan de DB-API

    """

    if password is None:
        password = getpass.getpass()  # Genereer vakje voor wachtwoord in te geven
 

In [5]:
display(Markdown(parse_markdown(introduction_file, section_number=4)))

## 1.3 Kolomnamen en input parameters

We leggen op voorhand reeds de **kolomnamen van de oplossingen**, en de **naam en types van de inputparameters** vast. Hier moet je je dus aan houden en mag je dus niks aan wijzigen.

- Het aantal kolommen (en hun volgorde) van jullie oplossing en de onze moeten exact overeen komen, vandaar dat de kolomnamen hieronder gegeven zijn. Dit komt trouwens van pas bij het opstellen van je queries! 
    
- Dankzij de voorbeeldparameters, die al gegeven zijn in de functiedefinitie e.g.: `query_42(connection, col_names, super_voorbeeldparam = 101)` weten jullie exact welke vorm en type (integer, lijst, etc) de inputparameters moeten hebben. Wijzig zeker niets aan de naam van die parameters (*super_voorbeeldparam* blijft *super_voorbeeldparam*). De default waarden geven op hun beurt dan weer informatie over het type van de inputparameters. Let wel: de (soms onnozele) default waarden zijn _louter ter illustratie_, zorg ervoor dat je query ook met andere waarden dan de defaults werkt!

**Samengevat: oplossingen die deze vorm niet respecteren, zullen crashen op onze machines en resulteren in een score van 0 op die query.**


In [6]:
# Inspecteer de dictionairy van kolomnamen

filename = os.path.join(solution_dir, 'all_q_colnam.json')
col_names = json.load(open(filename, 'r'))    

col_names # Inspecteer dictionary

{'query_01': ['tname', 'year', 'HomeRun'],
 'query_02': ['nameFirst', 'nameLast', 'birthYear', 'birthMonth', 'birthDay'],
 'query_03': ['nameFirst', 'nameLast', 'tname'],
 'query_04': ['teamID', 'name', 'yearID', 'W', 'L', 'MaxSalary'],
 'query_05': ['teamID', 'name', 'yearID', 'rank'],
 'query_06': ['playerID', 'nameFirst', 'nameLast'],
 'query_07': ['playerID', 'nameFirst', 'nameLast', 'teamID'],
 'query_08': ['teamID', 'tname', 'bestyear', 'HR', 'W'],
 'query_09': ['teamID', 'tname', 'yearID', 'rank', 'HR'],
 'query_10': ['teamId', 'HOFplayercount']}

In [7]:
display(Markdown(parse_markdown(instructions_file, section_number=[1,2])))

# 2. Instructies bij gebruik van de notebook

Hieronder volgen 10 onvolledige functies (e.g., `query_42(connection, col_names, super_voorbeeldparam = ['joske', 'jef'])`). Aan jullie om ze aan te vullen zodat de functie:  

1. Een corecte query opstelt
2. De query uitvoert op de database
3. Het resultaat teruggeeft in een DataFrame.

Voor stap 2 en 3 zijn de nodige functies al voorhanden, i.e.: `run_query(connection, query)` en `res_to_df(res, column_names)`. Jullie werk zal dus vooral bestaan uit stap 1, queries opstellen. Elke functie heeft minstens 2 inputargumenten:

1. `connection`:   Een connection object 
2. `column_names`: De kolomnamen van het Pandas DataFrame
    
Gevolgd door eventuele extra argumenten (e.g., `super_voorbeeldparam = ['joske','jef']`) die dienen om parameters in te query te injecteren. 

**Nogmaals: verander niets aan de namen van de functies, namen van de functie-argumenten en de kolomnamen van de resulterende DataFrames. Wijzigingen hieraan leiden onvermijdelijk tot een score van 0 op die query.**

Je kan naar believen extra cellen toevoegen om je queries te testen, resultaten te inspecteren etc. Daar dient deze notebook immers voor, deze notebook wordt **niet ingediend** als oplossing.

We vragen jullie om de finale, ingevulde functies te kopiëren naar een extern python script dat _enkel en alleen_ deze ingevulde functies bevat. Cf. de laatste sectie van deze notebook voor instructies omtrent hoe in te dienen.

## 2.1 Voorbeeld-query opstellen

Om jullie al wat op weg te zetten volgt hier een voorbeeldje over hoe je te werk kan gaan.

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame met teamnaam, jaar en aantal homeruns van teams die meer dan een gegeven aantal `homeruns` hadden in dat jaar.

Sorteer aflopend op aantal homeruns.

**Oplossing**



In [8]:
# Voorbeeld-query (oplossing)
def query_EX(connection, column_names, homeruns=20):
    # Bouw je query
    query="""
    select    t.name, t.yearID, t.HR
    from      Teams as t
    where     t.HR > {}
    order by  t.HR DESC;
    """.format(homeruns) # TIP: Zo krijg je parameters in de string (samen met `{}` in de string)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [9]:
display(Markdown(parse_markdown(instructions_file, section_number=3)))

## 2.2 Voorbeeld-query runnen

Om een query te runnen maken we gebruik van de hulpfunctie die we eerder ter beschikking stelden (e.g. `verbind_met_GB`). Concreet bestaat dit proces uit twee stappen:

1. Eerst maken we een verbindingsobject met de databank
2. Vervolgens runnen we onze query, en inspecteren we het resultaat.


In [10]:
# STAP 01: Verbindingsobject aanmaken

username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'db-project'          # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db, password='')

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (111 Connection refused)

In [ ]:
# STAP 02: Query runnen, resultaat ophalen en inspecteren.

kolomnamen_voorbeeldquery = col_names['query_01']        # Voorbeeldquery heeft dezelfde kolomnamen als query 1.

df = query_EX(c,kolomnamen_voorbeeldquery,homeruns=10)   # Query-functie uitvoeren, geeft resultaat van de query in een DataFrame

df.head()                                                # Inspecteer eerste resultaten (laat. head() weg om alles te zien)

## 3. Queries 

Hieronder vinden jullie de modeloplossingen. Door inconsistenties in de gegevensbank worden bij sommige queries meerdere oplossingen als correct aanzien. 

Bij correctie wordt jullie oplossing vergeleken met elke acceptable oplossing en de hoogste score telt.

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=1)))

**Oplossing**

In [ ]:
qry_01 = collect_relevant_solutions(ms, query_id=1)

# Print oplossing: in dit geval de source code van de geimporteerde python functies uit de `model_solution.py` file
for naam, functie, source_code in qry_01:  
    print(source_code) 

**Uitvoering (voorbeeld)**

Om een oplossing zelf te runnen, net zoals jullie zelf al deden in de opgave, kan je simpelweg de geimporteerde functies zelf gebruiken. Natuurlijk moet je nog wel de correcte parameters meegeven. 

De functie `collect_relevant_solutions` geeft een lijst van tupels terug. Zo'n tupel heeft de volgende structuur `(naam,  functie, source_code)`. Om de oplossing te runnen, heb je dus de `functie` nodig uit zo'n tupel. 

Bijvoorbeeld:

In [ ]:
"""
Stop de eerste functie in een variabele `qry`

    1. De eerste index (=0) verwijst naar de eerste mogelijke oplossing (soms zijn er meer)
    2. De tweede index (=1) verwijst naar het tweede element van de tupels die `collect_relevant_solutions` teruggeeft. Die
       zijn van de vorm (naam, functie, source_code)
"""

query_01_sol = qry_01[0][1]                   # Haal de functie eruit, geef nieuwe naam (`query_01_sol`)

query_01_sol(c, col_names['query_01']).head() # Deze functie kan je nu gewoon gebruiken alsof je ze zelf gedefinieerd zou hebben!

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=2)))

**Oplossing**

In [ ]:
qry_02 = collect_relevant_solutions(ms, query_id=2)

# Print solution(s)
for name, query, source_code in qry_02:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=3)))

**Oplossing**

In [ ]:
qry_03 = collect_relevant_solutions(ms, query_id=3)

# Print solution(s)
for name, query, source_code in qry_03:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=4)))

**Oplossing**

In [ ]:
qry_04 = collect_relevant_solutions(ms, query_id=4)

# Print solution(s)
for name, query, source_code in qry_04:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=5)))


**Oplossing**

In [ ]:
qry_05 = collect_relevant_solutions(ms, query_id=5)

# Print solution(s)
for name, query, source_code in qry_05:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=6)))

**Oplossing**

In [ ]:
qry_06 = collect_relevant_solutions(ms, query_id=6)

# Print solution(s)
for name, query, source_code in qry_06:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=7)))

**Oplossing**

In [ ]:
qry_07 = collect_relevant_solutions(ms, query_id=7)

# Print solution(s)
for name, query, source_code in qry_07:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=8)))

**Oplossing**

In [ ]:
qry_08 = collect_relevant_solutions(ms, query_id=8)

# Print solution(s)
for name, query, source_code in qry_08:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=9)))

**Oplossing**

In [ ]:
qry_09 = collect_relevant_solutions(ms, query_id=9)

# Print solution(s)
for name, query, source_code in qry_09:
    print(source_code)

In [ ]:
display(Markdown(parse_markdown(qry_description_file, section_number=10)))

**Oplossing**

In [ ]:
qry_10 = collect_relevant_solutions(ms, query_id=10)

# Print solution(s)
for name, query, source_code in qry_10:
    print(source_code)